- Alan Delon 
- C-Jovem Uece 
- Ciência de Dados - Intermediário

# Oficina 2: Etapas da mineração

# Introdução:

**Quais tipos de dados seriam necessários na fase de “Entendimento dos Dados” para alcançar esse objetivo?**

Do ponto de vista do negócio, o projeto tem como objetivo identificar padrões de 
compras e de comportamento dos clientes por meio dos dados gerados nos momentos das compras ( ( Dataset df_compras possui informações sobre Idade, Gênero, Tipo de promoção e Compra efetivada ou não ) e na informações das campanhas de marketing(Dataset data_promocoes contém Tipo de Promoção, Canal, Orçamento e Taxa de Retorno).

Questões a serem respondidas:

- **Quais características dos clientes que mais respondem as promoções?**
- **Quais as características das promoções que mais possuem taxa de retorno?**

A fim de responder essas perguntas duas perguntas foi utilizado a regra de associações Apriore e o modelo Random Forest, respectivamente.
Com isso será possível criar estratégias de marketing mais personalizadas para os 
clientes personalizando as promoções com caracterÌsticas mais relevantes.


In [15]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

data_compras = {
    'Idade': ['Jovem', 'Adulto', 'Idoso', 'Jovem', 'Adulto'],
    'Gênero': ['Masculino', 'Feminino', 'Masculino', 'Feminino', 'Masculino'],
    'Tipo_Promoção': ['Desconto', 'Brinde', 'Desconto', 'Brinde', 'Desconto'],
    'Compra': ['Sim', 'Não', 'Sim', 'Não', 'Sim']
}
data_promocoes = {
    'Tipo_Promoção': ['Desconto', 'Brinde', 'Desconto', 'Brinde', 'Desconto'],
    'Canal': ['Email', 'SMS', 'Email', 'SMS', 'Email'],
    'Orçamento': [500, 300, 700, 400, 600],
    'Taxa_Retorno': ['Alta', 'Baixa', 'Alta', 'Baixa', 'Alta']
}

df_compras = pd.DataFrame(data_compras)
df_promocoes = pd.DataFrame(data_promocoes)


# Preparação de dados:
    
   **Que procedimentos você seguiria na “Preparação dos Dados” para garantir que eles estejam prontos para a análise?**

Para o treinamento da regra Apriori e do modelo Radom Forest e foi necesário aplicar One-Hot-Encoding, que transforma todas as tipos de categorias em colunas de valores binários. Esse tratamento foi necessário pois os algoritmos lidam somente com tipos de dados numéricos. 

In [16]:
df_encoded = pd.get_dummies(df_compras)
df_encoded.head()

,Idade_Adulto,Idade_Idoso,Idade_Jovem,Gênero_Feminino,Gênero_Masculino,Tipo_Promoção_Brinde,Tipo_Promoção_Desconto,Compra_Não,Compra_Sim
0,False,False,True,False,True,False,True,False,True
1,True,False,False,True,False,True,False,True,False
2,False,True,False,False,True,False,True,False,True
3,False,False,True,True,False,True,False,True,False
4,True,False,False,False,True,False,True,False,True


In [17]:
label_encoder = LabelEncoder()
df_promocoes['Tipo_Promoção'] = label_encoder.fit_transform(df_promocoes['Tipo_Promoção'])
df_promocoes['Canal'] = label_encoder.fit_transform(df_promocoes['Canal'])
df_promocoes['Taxa_Retorno'] = label_encoder.fit_transform(df_promocoes['Taxa_Retorno'])

X = df_promocoes[['Tipo_Promoção', 'Canal', 'Orçamento']]
y = df_promocoes['Taxa_Retorno']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinamento: 

   **Quais algoritmos ou técnicas você usaria na “Modelagem” para extrair padrões e insights dos dados?**

## Apriori
A regra de associação Apriori foi utilizada para encontrar as caracteríticas mais comumns em compradores e o tipo de promoção que gerou mais compras.

In [18]:
frequent_itemsets = apriori(df_encoded, min_support=0.2, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
df1 = pd.DataFrame(rules)
print(rules[['antecedents', 'consequents', 'support', 'confidence']])

                                          antecedents  ... confidence
0                                       (Idade_Idoso)  ...        1.0
1                                       (Idade_Idoso)  ...        1.0
2                                       (Idade_Idoso)  ...        1.0
3                              (Tipo_Promoção_Brinde)  ...        1.0
4                                   (Gênero_Feminino)  ...        1.0
..                                                ...  ...        ...
86        (Compra_Sim, Idade_Jovem, Gênero_Masculino)  ...        1.0
87  (Tipo_Promoção_Desconto, Idade_Jovem, Gênero_M...  ...        1.0
88                          (Compra_Sim, Idade_Jovem)  ...        1.0
89              (Tipo_Promoção_Desconto, Idade_Jovem)  ...        1.0
90                    (Idade_Jovem, Gênero_Masculino)  ...        1.0

[91 rows x 4 columns]


In [19]:
SELECT antecedents, consequents, support, confidence
FROM df1;

,antecedents,consequents,support,confidence
0,frozenset({'Idade_Idoso'}),frozenset({'Gênero_Masculino'}),0.2,1.0
1,frozenset({'Idade_Idoso'}),frozenset({'Tipo_Promoção_Desconto'}),0.2,1.0
2,frozenset({'Idade_Idoso'}),frozenset({'Compra_Sim'}),0.2,1.0
3,frozenset({'Tipo_Promoção_Brinde'}),frozenset({'Gênero_Feminino'}),0.4,1.0
4,frozenset({'Gênero_Feminino'}),frozenset({'Tipo_Promoção_Brinde'}),0.4,1.0
...,...,...,...,...
86,"frozenset({'Compra_Sim', 'Idade_Jovem', 'Gêner...",frozenset({'Tipo_Promoção_Desconto'}),0.2,1.0
87,"frozenset({'Tipo_Promoção_Desconto', 'Idade_Jo...",frozenset({'Compra_Sim'}),0.2,1.0
88,"frozenset({'Compra_Sim', 'Idade_Jovem'})","frozenset({'Tipo_Promoção_Desconto', 'Gênero_M...",0.2,1.0
89,"frozenset({'Tipo_Promoção_Desconto', 'Idade_Jo...","frozenset({'Compra_Sim', 'Gênero_Masculino'})",0.2,1.0


## Radom Forest

O modelo foi utilizado para identificar quais elementos das campanhas de marketing mais impactam no retorno.

In [20]:
# Treinando o modelo Random Forest
modelo = RandomForestClassifier(random_state=42)
modelo.fit(X_train, y_train)

# Fazendo previsões
y_pred = modelo.predict(X_test)


# Conclusão:
**Como você avaliaria os resultados para garantir que eles estão alinhados com os objetivos do negócio antes de implementá-los para gerar valor real?**


A fim de garantir que o modelo está atendendo aos objetivos estabelecidos, se faz 
necessário a definiçãoo de indicadores de desempenho. Isso também irá permitir a
constante atualizaçãoo, manutenção e melhoramento do modelo. Os principais 
parâmetros observados inicialmente seráo a precisão, acurácia, F1-Score e suporte, que também pode ser extraído utilizando os scikit-learn.

In [21]:
importances = modelo.feature_importances_
for feature, importance in zip(X.columns, importances):
    print(f"{feature}: {importance:.2f}")
    
# Avaliando o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))


Tipo_Promoção: 0.36
Canal: 0.34
Orçamento: 0.30
Acurácia: 1.0
Relatório de Classificação:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

